# Opioid Data Cleaning
### Date: 10/31/2021  
<br />
    
## Step 1: Load In Top 50K rows

In [13]:
import numpy as np
import pandas as pd
from tqdm import tqdm


In [15]:
# This is a very big file, so we will first load in a few lines to explore the structure
head = pd.read_csv(
    "C:/Users/rober/Desktop/arcos_all_washpost.tsv.gz",
    compression="gzip",
    sep="\t",
    nrows=50000,
)

# See what columns we have
# head.sample(10)


C:\Users\rober\miniconda3\envs\general\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,5,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
# Keep these columns
# keep = ['TRANSACTION_DATE', 'BUYER_STATE', 'BUYER_COUNTY', 'BUYER_ZIP', 'DRUG_CODE', 'DRUG_NAME', 'CALC_BASE_WT_IN_GM', 'DOSAGE_UNIT', 'MME_Conversion_Factor', 'dos_str']
# keeping 'CALC_BASE_WT_IN_GM', 'DOSAGE_UNIT', 'MME_Conversion_Factor', 'dos_str' to later help us decide which column is the best representation of shipment
head = head[
    [
        "TRANSACTION_DATE",
        "BUYER_STATE",
        "BUYER_COUNTY",
        "BUYER_ZIP",
        "DRUG_CODE",
        "DRUG_NAME",
        "CALC_BASE_WT_IN_GM",
        "DOSAGE_UNIT",
        "MME_Conversion_Factor",
        "dos_str",
    ]
]



In [17]:
# Convert TRANSACTION_DATE column to a usable format

# The TRANSACTION_DATE column is in the format of (m)mddyyyy
# Months are not zero-padded, and days are zero-padded
# As such, 1022020 means January 2nd, 2020 rather than October 2nd, 2020. In other words, it's 1/02/2020

# Convert the TRANSACTION_DATE column to yyyy/(m)m/dd
head["TRANSACTION_DATE"] = (
    head["TRANSACTION_DATE"].astype(str).str[-4:]
    + "/"
    + head["TRANSACTION_DATE"].astype(str).str[:-6]
    + "/"
    + head["TRANSACTION_DATE"].astype(str).str[-6:-4]
)
# Convert to datetime format
head["TRANSACTION_DATE"] = pd.to_datetime(head["TRANSACTION_DATE"], format="%Y/%m/%d")

# add a year column
head["TRANSACTION_YEAR"] = head["TRANSACTION_DATE"].astype("datetime64[Y]")
# add a month column
head["TRANSACTION_MONTH"] = head["TRANSACTION_DATE"].astype("datetime64[M]")

In [18]:
# Zip code needs to be 5-digits
# For zipcode that are not 5-digits, pad with 0s at the beginning
head["BUYER_ZIP"] = head["BUYER_ZIP"].astype(str)
head["BUYER_ZIP"] = head["BUYER_ZIP"].str.pad(width=5, side="left", fillchar="0")


### Step 1.1 Explore Null Values & Decide Which Column to Use to Represent Opioid Shipment

In [22]:
head.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   TRANSACTION_DATE       50000 non-null  datetime64[ns]
 1   BUYER_STATE            50000 non-null  object        
 2   BUYER_COUNTY           49998 non-null  object        
 3   BUYER_ZIP              50000 non-null  object        
 4   DRUG_CODE              50000 non-null  int64         
 5   DRUG_NAME              50000 non-null  object        
 6   CALC_BASE_WT_IN_GM     50000 non-null  float64       
 7   DOSAGE_UNIT            49999 non-null  float64       
 8   MME_Conversion_Factor  50000 non-null  float64       
 9   dos_str                49914 non-null  float64       
 10  TRANSACTION_YEAR       50000 non-null  datetime64[ns]
 11  TRANSACTION_MONTH      50000 non-null  datetime64[ns]
 12  shipment_quantity      49913 non-null  float64       
dtypes

In [7]:
# Why are there nulls for county?
head.loc[
    head["BUYER_COUNTY"].isna() == True,
]
# I looked up the zip code 02401. It's an inactive zip code
# It used to belong to Brockton, MA according to Google
# However, USPS doesn't have this zip code anymore - it might have gone inactive

# If there are only a small proportion of rows with this issue, we can drop those rows
# Since only 2 out of the first 50K rows have this issue, it's possible that this is very rare, and we're safe to drop them


,TRANSACTION_DATE,BUYER_STATE,BUYER_COUNTY,BUYER_ZIP,DRUG_CODE,DRUG_NAME,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME_Conversion_Factor,dos_str,TRANSACTION_YEAR,TRANSACTION_MONTH
4035,2006-05-18,MA,NaN,02401,9143,OXYCODONE,0.959148,200.0,1.5,5.35,2006-01-01,2006-05-01
4036,2006-07-31,MA,NaN,02401,9143,OXYCODONE,5.378400,1200.0,1.5,5.00,2006-01-01,2006-07-01


In [8]:
# Why are there nulls for dos_str?
head.loc[
    head["dos_str"].isna() == True,
]

# Are all these drugs OXYCODONE?
# Yes
head.loc[
    (head["dos_str"].isna() == True) & (head["DRUG_NAME"] != "OXYCODONE"),
]

# Do all OXYCODONEs have NaN dos_str?
# No. A lot of OXYCODONE shipments have values for dos_str
head.loc[
    (head["dos_str"].isna() == False) & (head["DRUG_NAME"] == "OXYCODONE"),
]

# Are there other patterns?
# Most NaN dos_str occur in NY and PA
head.loc[
    head["dos_str"].isna() == True,
].BUYER_STATE.value_counts()
# No pattern regarding TRANSACTION_YEAR
head.loc[
    head["dos_str"].isna() == True,
].TRANSACTION_YEAR.value_counts()

# 86 / 50,000 rows have missing dos_str, giving a 0.172% missing rate
# Seems fine to drop the missing rows if the final decision is to use 'dos_str' for the analysis

2006-01-01    26
2008-01-01    20
2007-01-01    13
2010-01-01    12
2009-01-01     8
2011-01-01     7
Name: TRANSACTION_YEAR, dtype: int64

In [9]:
# Calculate opioid shipment quanity using this formula
# quantity = dos_str * DOSAGE_UNITS * MME_Conversion_Factor
head["shipment_quantity"] = head.dos_str * head.DOSAGE_UNIT * head.MME_Conversion_Factor


,TRANSACTION_DATE,BUYER_STATE,BUYER_COUNTY,BUYER_ZIP,DRUG_CODE,DRUG_NAME,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME_Conversion_Factor,dos_str,TRANSACTION_YEAR,TRANSACTION_MONTH
0,2012-12-26,MA,MIDDLESEX,02148,9193,HYDROCODONE,0.60540,100.0,1.0,10.0,2012-01-01,2012-12-01
1,2009-03-11,AZ,MARICOPA,85085,9193,HYDROCODONE,0.12108,40.0,1.0,5.0,2009-01-01,2009-03-01
2,2008-11-25,AZ,MARICOPA,85233,9193,HYDROCODONE,3.63240,1200.0,1.0,5.0,2008-01-01,2008-11-01
3,2009-06-12,AZ,MARICOPA,85233,9193,HYDROCODONE,2.72430,600.0,1.0,7.5,2009-01-01,2009-06-01
4,2009-10-02,AZ,MARICOPA,85233,9193,HYDROCODONE,0.90810,300.0,1.0,5.0,2009-01-01,2009-10-01


In [ ]:
# Do we have NAs? How many?
head[head["shipment_quantity"].isna() == True]

# We have 87 NAs out of 50,000. All NAs caused by NA dos_str
# This seems fine

# Drop NA shipment_quantity
head = head[head["shipment_quantity"].notna()]


### Step 1.2: Group by county, state, year

In [10]:
# Opioid shipment per county per year
annual = head.groupby(['TRANSACTION_YEAR', 'BUYER_COUNTY', 'BUYER_STATE'], as_index=False)['shipment_quantity'].sum()

# Opioid shipment per county per month
monthly = head.groupby(['TRANSACTION_MONTH', 'BUYER_COUNTY', 'BUYER_STATE'], as_index=False)['shipment_quantity'].sum()

## Step 2: Load In Full Data Set

In [34]:
# Read in full data set
opioid = pd.DataFrame()
annual_total = pd.DataFrame()
monthly_total = pd.DataFrame()

for chunk in tqdm(
    pd.read_csv(
        "C:/Users/rober/Desktop/arcos_all_washpost.tsv.gz",
        compression="gzip",
        sep="\t",
        header=0,
        usecols=[
            "TRANSACTION_DATE",
            "BUYER_STATE",
            "BUYER_COUNTY",
            "BUYER_ZIP",
            "DRUG_CODE",
            "DRUG_NAME",
            "CALC_BASE_WT_IN_GM",
            "DOSAGE_UNIT",
            "MME_Conversion_Factor",
            "dos_str",
        ],
        dtype={
            "TRANSACTION_DATE": np.int32,
            "BUYER_STATE": object,
            "BUYER_COUNTY": object,
            "BUYER_ZIP": object,
            "DRUG_CODE": np.int32,
            "DRUG_NAME": object,
            "CALC_BASE_WT_IN_GM": np.float32,
            "DOSAGE_UNIT": np.float32,
            "MME_Conversion_Factor": np.float16,
            "dos_str": np.float16,
        },
        iterator=True,
        chunksize=3500000,
    )
):
    # Correct TRANSACTION_DATE
    # Convert the TRANSACTION_DATE column to yyyy/(m)m/dd
    chunk["TRANSACTION_DATE"] = (
        chunk["TRANSACTION_DATE"].astype(str).str[-4:]
        + "/"
        + chunk["TRANSACTION_DATE"].astype(str).str[:-6]
        + "/"
        + chunk["TRANSACTION_DATE"].astype(str).str[-6:-4]
    )
    # Convert to datetime format
    chunk["TRANSACTION_DATE"] = pd.to_datetime(
        chunk["TRANSACTION_DATE"], format="%Y/%m/%d"
    )
    # add a year column
    chunk["TRANSACTION_YEAR"] = chunk["TRANSACTION_DATE"].astype("datetime64[Y]")
    # add a month column
    chunk["TRANSACTION_MONTH"] = chunk["TRANSACTION_DATE"].astype("datetime64[M]")

    # Correct zip code
    # pad zip codes with 0s to make them 5-digit long
    chunk["BUYER_ZIP"] = chunk["BUYER_ZIP"].astype(str)
    chunk["BUYER_ZIP"] = chunk["BUYER_ZIP"].str.pad(width=5, side="left", fillchar="0")

    # Calculate opioid shipment quanity using this formula
    # quantity = dos_str * DOSAGE_UNITS * MME_Conversion_Factor
    chunk["shipment_quantity"] = chunk.dos_str * chunk.DOSAGE_UNIT * chunk.MME_Conversion_Factor
    # Drop NA shipment_quantity
    chunk = chunk[chunk["shipment_quantity"].notna()]

    # Group by to get annual and monthly shipments
    # Opioid shipment per county per year
    annual = chunk.groupby(
        ["TRANSACTION_YEAR", "BUYER_COUNTY", "BUYER_STATE"], as_index=False
    )["shipment_quantity"].sum()
    # Opioid shipment per county per month
    monthly = chunk.groupby(
        ["TRANSACTION_MONTH", "BUYER_COUNTY", "BUYER_STATE"], as_index=False
    )["shipment_quantity"].sum()

    # append
    # opioid = opioid.append(chunk)
    annual_total = annual_total.append(annual)
    monthly_total = monthly_total.append(monthly)

# save data to parquet
annual_total.to_parquet(
    "../20_intermediate_files/opioid_annual_total.parquet", engine="fastparquet"
)
monthly_total.to_parquet(
    "../20_intermediate_files/opioid_monthly_total.parquet", engine="fastparquet"
)


52it [18:33, 21.41s/it]


In [35]:
# We need to group by again in case there are ungrouped rows as we appended
annual_total = annual_total.groupby(
    ["TRANSACTION_YEAR", "BUYER_COUNTY", "BUYER_STATE"], as_index=False
)["shipment_quantity"].sum()
monthly_total = monthly_total.groupby(
    ["TRANSACTION_MONTH", "BUYER_COUNTY", "BUYER_STATE"], as_index=False
)["shipment_quantity"].sum()


In [36]:
# Ensure we don't have duplicates
assert not annual_total[["TRANSACTION_YEAR", "BUYER_COUNTY", "BUYER_STATE"]].duplicated().any()
assert not monthly_total[["TRANSACTION_MONTH", "BUYER_COUNTY", "BUYER_STATE"]].duplicated().any()

In [37]:
# save data again
annual_total.to_parquet(
    "../20_intermediate_files/opioid_annual_total.parquet", engine="fastparquet"
)
monthly_total.to_parquet(
    "../20_intermediate_files/opioid_monthly_total.parquet", engine="fastparquet"
)


## Step 3: Basic EDA

In [38]:
# Load data
annual_total = pd.read_parquet("../20_intermediate_files/opioid_annual_total.parquet")
monthly_total = pd.read_parquet("../20_intermediate_files/opioid_monthly_total.parquet")

In [39]:
annual_total

,TRANSACTION_YEAR,BUYER_COUNTY,BUYER_STATE,shipment_quantity
0,2006-01-01,ABBEVILLE,SC,4.160439e+06
1,2006-01-01,ACADIA,LA,2.954760e+07
2,2006-01-01,ACCOMACK,VA,5.289719e+06
3,2006-01-01,ADA,ID,1.204424e+08
4,2006-01-01,ADAIR,IA,1.478750e+06
...,...,...,...,...
21575,2012-01-01,YUBA,CA,5.906503e+07
21576,2012-01-01,YUMA,AZ,7.070816e+07
21577,2012-01-01,YUMA,CO,2.906652e+06
21578,2012-01-01,ZAPATA,TX,1.512700e+06


In [40]:
# Add a COUNTY_STATE column to help us count the number of distinct counties more easily
# Because counties can have the same name across states
annual_total['COUNTY_STATE'] = annual_total['BUYER_COUNTY'] + ',' + annual_total['BUYER_STATE']
monthly_total['COUNTY_STATE'] = monthly_total['BUYER_COUNTY'] + ',' + monthly_total['BUYER_STATE']

In [41]:
# How many counties do we have?
    # 3119
annual_total[['COUNTY_STATE']].drop_duplicates()

# What about by year?
annual_total.groupby(['TRANSACTION_YEAR'])['COUNTY_STATE'].nunique()

# What about by month?
monthly_total.groupby(['TRANSACTION_MONTH'])['COUNTY_STATE'].nunique()

TRANSACTION_MONTH
2006-01-01    3060
2006-02-01    3067
2006-03-01    3052
2006-04-01    3062
2006-05-01    3060
              ... 
2012-08-01    3068
2012-09-01    3074
2012-10-01    3075
2012-11-01    3070
2012-12-01    3072
Name: COUNTY_STATE, Length: 84, dtype: int64